# 記憶を持つエージェント

# 準備

In [6]:
pip install langchain_community

In [9]:
import getpass
import os
api_key = getpass.getpass("OpenAI API キーを入力してください: ")
os.environ["OPENAI_API_KEY"] = api_key

api_key1 = getpass.getpass("SERPAPI API キーを入力してください: ")
os.environ["SERPAPI_API_KEY"] = api_key1

OpenAI API キーを入力してください:  ········
SERPAPI API キーを入力してください:  ········


# プロンプトの定義

In [15]:
from langchain_core.prompts import PromptTemplate

input_variables=['agent_scratchpad', 'input', 'tool_names', 'tools']
template="""\
Answer the following questions as best you can. You have access to the following tools:

{tools}

Use the following format:

Question: the input question you must answer
Thought: you should always think about what to do
Action: the action to take, should be one of [{tool_names}]
Action Input: the input to the action
Observation: the result of the action
... (this Thought/Action/Action Input/Observation can repeat N times)
Thought: I now know the final answer
Final Answer: the final answer to the original input question

Begin!

Previous conversation history: {chat_history}
Question: {input}
Thought:{agent_scratchpad}"""

prompt = PromptTemplate(input_variables=input_variables, template=template)
print(prompt.template)

Answer the following questions as best you can. You have access to the following tools:

{tools}

Use the following format:

Question: the input question you must answer
Thought: you should always think about what to do
Action: the action to take, should be one of [{tool_names}]
Action Input: the input to the action
Observation: the result of the action
... (this Thought/Action/Action Input/Observation can repeat N times)
Thought: I now know the final answer
Final Answer: the final answer to the original input question

Begin!

Previous conversation history: {chat_history}
Question: {input}
Thought:{agent_scratchpad}


# 会話ごとに記憶を切り替える

In [18]:
from langchain.memory import ChatMessageHistory

store = {}

def get_by_session_id(session_id: str) -> ChatMessageHistory:
    if session_id not in store:
        store[session_id] = ChatMessageHistory()
    return store[session_id]

# メッセージ履歴を追加

In [21]:
from langchain_openai import ChatOpenAI
from langchain.agents import load_tools
from langchain.agents import AgentExecutor, create_react_agent
from langchain_core.runnables.history import\
RunnableWithMessageHistory

model = ChatOpenAI(model="gpt-4o-mini")
tools = load_tools(["serpapi"], llm=model)
agent = create_react_agent(model, tools, prompt)
agent_executor = AgentExecutor(agent=agent, tools=tools,\
verbose=True)

agent_with_chat_history = RunnableWithMessageHistory(
    agent_executor,
    get_by_session_id,
    input_messages_key="input",
    history_messages_key="chat_history",
)

response = agent_with_chat_history.invoke({"input": "株式会社\
Elithの住所を教えてください。最新の公式情報として公開されているものを教え\
てください。"},
    config={"configurable": {"session_id": "test-session1"}})



> Entering new AgentExecutor chain...
株式会社Elithの住所に関する最新の公式情報を確認する必要があります。公式サイトや最新のプレスリリースなどを検索するのが良さそうです。  
Action: Search  
Action Input: 株式会社Elith 住所 公式サイト  ['株式会社エリスの公式HPです。 Elithは、最先端のAI技術をビジネス実装し価値を生み出す松尾研発スタートアップです.', '\u200bElith OFFICE · 東京都文京区本郷3-30-10本郷 K&Kビル1F · 東京メトロ丸ノ内線 / 本郷三丁目駅 徒歩約2分 都営大江戸線 / 本郷三丁目駅 徒歩約3分 東京メトロ千代田線 / ...', '会社概要株式会社Elithは、東大松尾研発スタートアップ(R)として、最先端のAI ... 住所. 日本113-0033東京都文京区本郷3-30-10本郷 K&Kビル1F. この出展社を見ている ...', '連絡先. https://www.elith.co.jp/ contact@elith.co.jp. 住所. 日本113-0033東京都文京区本郷3-30-10本郷 K&Kビル1F. この出展社を見ている人は、こちらの出展社も見てい ...', '法人番号: 6011001151296 ; 法人名: 株式会社Elith ; 法人名ふりがな: えりす ; 法人名英語 ; 本店所在地: 東京都文京区本郷3丁目30番10号本郷K＆Kビル1F ...', '東大 松尾研発スタートアップ®の株式会社Elith と株式会社ニューテックが協業を開始. 2025年2月17日 19時00分. Elith執筆『LLMエージェントの最新事情』 ...', '株式会社Elithの概要. 住所, 〒 〒113-0033 東京都文京区本郷3-30-10本郷 K&Kビル1F. 分野, 映像認識・画像解析（視覚的）, 音声・音による入力（聴覚的）. 業界, 医療 ...', '- 会社名：株式会社Elith · - 代表者：若林怜帆人（代表取締役）、井上顧基（取締役CEO兼CTO） · - 所在地：東京都渋谷区恵比寿西2-3-13-601 · - 会社ウェブサイト：Elithの ...', '"AIの「今」をON AI

In [23]:
print(get_by_session_id("test-session1"))

Human: 株式会社Elithの住所を教えてください。最新の公式情報として公開されているものを教えてください。
AI: 株式会社Elithの住所は、〒113-0033 東京都文京区本郷3-30-10 本郷K&Kビル1Fです。


In [25]:
response = agent_with_chat_history.invoke({"input": "先ほど尋ね\
た会社は何の会社ですか？"},
    config={"configurable": {"session_id": "test-session1"}})



> Entering new AgentExecutor chain...
株式会社Elithについて調べる必要がある。  
Action: Search  
Action Input: "株式会社Elith 会社概要"  [' entity_type: related_questions.', '事業内容：AI（人工知能）に関する研究、開発、設計、企画、教育、販売、保守、コンサルティング業務所在地：東京都渋谷区恵比寿西2-3-13-601\u200b', '株式会社エリスの公式HPです。 Elithは、最先端のAI技術をビジネス実装し価値を生み出す松尾研発スタートアップです.', '会社概要株式会社Elithは、東大松尾研発スタートアップ(R)として、最先端のAI技術を活用しビジネスに新たな価値を提供しています。国際学会発表・書籍出版の学術実績と ...', '株式会社Elithの魅力を伝えるコンテンツと、住所や代表・従業員などの会社情報です。ElithはAIのもっと身近な活用を推進して、より暮らしやすい世の中にします。', '会社概要株式会社Elithは、東大松尾研発スタートアップ(R)として、最先端のAI技術を活用しビジネスに新たな価値を提供しています。国際学会発表・書籍出版の学術実績と ...', '製造・金融・医療業等向けに、生成AI利活用・LLM、画像AI開発・システム構築、AI教育アドバイザリー等のソリューションを提供。 決算書作成RPA対応AIを活用した自社株 ...', '株式会社Elithの概要. 住所, 〒 〒113-0033 東京都文京区本郷3-30-10本郷 K&Kビル1F. 分野, 映像認識・画像解析（視覚的）, 音声・音による入力（聴覚的）. 業界, 医療 ...', '株式会社Elith · 事業内容 · 株主. Not Available. 49 百万円. 評価額. 5 百万円. 累計調達額. 9999人. ログインして見る. 社員数 前月 ...', '高度な画像解析技術で半導体部品の微細な異常を検知し、効率的な欠陥識別と製品品質向上をサポートします。また、異常検知結果を活用し、生産プロセスを最適化して歩留りを ...', '東大・松尾研発スタートアップの株式会社Elith(エリス)の公式アカウントです。 AI開発・研究を行っ

In [27]:
print(get_by_session_id("test-session1"))

Human: 株式会社Elithの住所を教えてください。最新の公式情報として公開されているものを教えてください。
AI: 株式会社Elithの住所は、〒113-0033 東京都文京区本郷3-30-10 本郷K&Kビル1Fです。
Human: 先ほど尋ねた会社は何の会社ですか？
AI: 株式会社Elithは、AI技術を活用してビジネスに新たな価値を提供するスタートアップです。


In [29]:
print(get_by_session_id("test-session2"))

In [31]:
response = agent_with_chat_history.invoke({"input": "先ほど尋ね\
た会社は何の会社ですか？"},
    config={"configurable": {"session_id": "test-session2"}})



> Entering new AgentExecutor chain...
会社についての具体的な情報が必要ですので、その会社の名前や業種などの詳細がわからないと分析できません。もしその情報を知っている場合は教えてください。そうでない場合は、一般的にその会社の業界や製品について調べる必要があります。そこで、会社名を特定するために検索を行います。

Action: Search
Action Input: 先ほど尋ねた会社は何の会社ですか？
[' entity_type: related_questions.', '例えば、ちょっと仲良くなった人とか、習い事の人とか・・・に、あまり言いたくないんです。 例えば、「あのへんの会社でOLしてます」でいいでしょうか。', '「お聞きしました」と「伺いました」の直訳的な意味. 「お聞きしました」は文字通り「聞いた」という意味で、自分自身が何かを聞いたときに用いる表現です ...', '取引先や顧客などに向けた内容をメールで送信する際には、自分の勤める会社のことを「弊社」と表現するのが基本です。 立場によって変わるケースがある ...', 'ずっと一緒に住んでいますか。 Fさんの仕事は何ですか。 Mさんも仕事しますか。 毎日、仕事ありません。ない？ じゃ、毎日、家にいますか。家で何をしますか。', '面接官を担当する方は知っておくべき、人柄を見抜く質問や、聞いてはいけないタブーな質問を例文を交え解説します。面接前で不安な方も是非ご覧ください。', '「訪ねる」の謙譲語である「伺う」は、ビジネスでもプライベートでも使われています。 また、「伺う」は相手に質問をする「尋ねる」の謙譲語でも ...', '会社訪問の質問で好印象を残すために、聞きたい内容の要点を明確にしておきましょう！ 今回は会社訪問の事前準備や質問時のマナーを紹介していきます。', '「おすすめは何ですか？」と聞かれたとき、感じのいい人は何と答える？ それを語るのは、「感じのいい人」に生まれ変われるとっておきのコツを紹介する ...', '「した方がいいですか」は丁寧語の疑問形を含み、相手への尊重と配慮を示す表現となっているので、目上の人にも使えます。ただし、相手に尋ねるまでもない ...', '電話対応が苦手と感じている人のために、実際の仕事で使える基本的な言葉